In [1]:
# Setup: project path and OMDb API key
import os
import sys
from pathlib import Path
import numpy as np

# Ensure project root is on sys.path so `import app` works when running from notebooks/
PROJECT_ROOT = Path.cwd().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Configure OMDb API key for experiments.
# Prefer setting OMDB_API_KEY in your environment or `.env` file.
# For quick experiments, you can set it here (replace the placeholder):
OMDB_API_KEY = os.environ.get("OMDB_API_KEY") or "YOUR_OMDB_API_KEY_HERE"
os.environ["OMDB_API_KEY"] = OMDB_API_KEY
print("OMDB_API_KEY set:", "present" if bool(os.environ.get("OMDB_API_KEY")) and os.environ.get("OMDB_API_KEY") != "YOUR_OMDB_API_KEY_HERE" else "missing")

OMDB_API_KEY set: present


In [2]:
# Imports, async runner, and loading the recommender
import asyncio
from typing import Any

from app.services.recommender import recommend_movies


def run(coro):
    """Run an async coroutine from a notebook cell, returning its result."""
    try:
        loop = asyncio.get_event_loop()
    except RuntimeError:
        loop = None
    if loop and loop.is_running():
        # If already in an event loop (e.g., IPython), create a new task
        return asyncio.ensure_future(coro)
    return asyncio.run(coro)


def show_results(items: list[dict[str, Any]]):
    if not items:
        print("No results.")
        return
    for i, m in enumerate(items, 1):
        print(f"{i}. {m.get('title')}  |  Rating: {m.get('vote_average')}  |  Released: {m.get('release_date')}")
        if m.get("overview"):
            print(f"   {m['overview'][:200]}{'...' if len(m['overview'])>200 else ''}")
        if m.get("poster_url"):
            print(f"   Poster: {m['poster_url']}")
        print()


In [3]:
# Example 1: Keyword strategy (default)
MOOD = "gory fun"
LIMIT = 6

movies = run(recommend_movies(mood=MOOD, limit=LIMIT, strategy="keyword"))

# If we're in a running loop (Jupyter), `run` may return a Task. Await it to get results.
try:
    from asyncio import Future
    if hasattr(movies, "__await__") or isinstance(movies, Future):
        movies = await movies
except Exception:
    pass

show_results(movies)


1. Amityville Horror: The Evil Escapes  |  Rating: 4.4  |  Released: 12 May 1989
   The demonic forces in the Amityville house transfer to an ancient lamp, which finds its way to a remote California mansion where the evil manipulates a little girl by manifesting itself in the form of...
   Poster: https://m.media-amazon.com/images/M/MV5BOWNkNGE1NzMtMThhZS00OWY5LWJjMDMtNzcyNjRkMTQ4MDBlXkEyXkFqcGc@._V1_SX300.jpg

2. A Classic Horror Story  |  Rating: 5.7  |  Released: 14 Jul 2021
   In this gruesome suspense film, strangers traveling in southern Italy become stranded in the woods, where they must fight desperately to get out alive.
   Poster: https://m.media-amazon.com/images/M/MV5BYTg5ZDgxNDAtMzFkNS00M2UwLWI4N2EtOTczYjM0MjYwZDRjXkEyXkFqcGc@._V1_SX300.jpg

3. Horror Express  |  Rating: 6.5  |  Released: 03 Jan 1974
   While on the Trans-Siberian Express, an anthropologist and his rival must contain the threat posed by the former's cargo: a prehistoric ape which is the host for a parasiti

In [4]:
# Example 2: Embedding/TF-IDF strategy
MOOD = "claustrophobic psychological"
LIMIT = 6

movies = run(recommend_movies(mood=MOOD, limit=LIMIT, strategy="embedding"))

try:
    from asyncio import Future
    if hasattr(movies, "__await__") or isinstance(movies, Future):
        movies = await movies
except Exception:
    pass

show_results(movies)


1. Amityville Horror: The Evil Escapes  |  Rating: 4.4  |  Released: 12 May 1989
   The demonic forces in the Amityville house transfer to an ancient lamp, which finds its way to a remote California mansion where the evil manipulates a little girl by manifesting itself in the form of...
   Poster: https://m.media-amazon.com/images/M/MV5BOWNkNGE1NzMtMThhZS00OWY5LWJjMDMtNzcyNjRkMTQ4MDBlXkEyXkFqcGc@._V1_SX300.jpg

2. House III: The Horror Show  |  Rating: 5.1  |  Released: 28 Apr 1989
   Detective McCarthy finally catches "Meat Cleaver Max", a serial killer, who promises revenge during his execution. Nonetheless, a parapsychologist tells the detective that the only hope of stopping Ma...
   Poster: https://m.media-amazon.com/images/M/MV5BODkxNWI2NWItMmU5MS00OWViLThhN2YtNDg1YTg4MzgyODQxXkEyXkFqcGc@._V1_SX300.jpg

3. In Search of Darkness: A Journey Into Iconic '80s Horror  |  Rating: 8.0  |  Released: 13 Oct 2019
   An exploration of '80s horror movies through the perspective of the actors

In [6]:
# Improved OMDb helpers: plot=full, multi-page search, type/year filters
import os
from typing import Any, Iterable
import httpx

OMDB_BASE_URL = "https://www.omdbapi.com/"


async def omdb_get(params: dict[str, Any]) -> dict[str, Any]:
    key = os.environ.get("OMDB_API_KEY") or ""
    merged = {"apikey": key}
    merged.update(params)
    async with httpx.AsyncClient(timeout=httpx.Timeout(12.0, connect=5.0)) as client:
        resp = await client.get(OMDB_BASE_URL, params=merged)
        resp.raise_for_status()
        data = resp.json()
    if isinstance(data, dict) and data.get("Response") == "False":
        return {}
    return data


async def search_ids(query: str, *, types: Iterable[str] = ("movie",), pages: int = 3, year: int | None = None) -> list[str]:
    ids: list[str] = []
    for t in types:
        for page in range(1, max(1, pages) + 1):
            params: dict[str, Any] = {"s": query, "type": t, "page": page}
            if year:
                params["y"] = year
            data = await omdb_get(params)
            results = data.get("Search") if isinstance(data, dict) else None
            for item in results or []:
                imdb_id = item.get("imdbID")
                if isinstance(imdb_id, str):
                    ids.append(imdb_id)
    # De-duplicate while preserving order
    return list(dict.fromkeys(ids))


async def get_detail_full(imdb_id: str) -> dict[str, Any]:
    return await omdb_get({"i": imdb_id, "plot": "full"})


async def fetch_details(ids: list[str], *, horror_only: bool = True) -> list[dict[str, Any]]:
    details: list[dict[str, Any]] = []
    for imdb_id in ids:
        d = await get_detail_full(imdb_id)
        if not d:
            continue
        genre = (d.get("Genre") or "").lower()
        if horror_only and "horror" not in genre:
            continue
        poster = d.get("Poster")
        poster_url = poster if poster and poster != "N/A" else None
        details.append(
            {
                "imdb_id": d.get("imdbID"),
                "type": (d.get("Type") or d.get("Type") or "movie"),
                "title": d.get("Title"),
                "overview": d.get("Plot") or "",
                "poster_url": poster_url,
                "release_date": d.get("Released"),
                "year": d.get("Year"),
                "vote_average": float(d.get("imdbRating") or 0) if (d.get("imdbRating") and d.get("imdbRating") != "N/A") else None,
                "imdb_votes_raw": d.get("imdbVotes"),
                "metascore_raw": d.get("Metascore"),
                "ratings": d.get("Ratings"),
                "awards": d.get("Awards"),
                "genre": d.get("Genre"),
                "language": d.get("Language"),
                "country": d.get("Country"),
            }
        )
    return details


In [7]:
# Scoring and filters: votes-aware score, year band, language, type
from math import log
from typing import Optional


def parse_int(s: Optional[str]) -> int:
    if not s:
        return 0
    try:
        return int(str(s).replace(",", "").strip())
    except Exception:
        return 0


def parse_float(s: Optional[str]) -> float:
    if not s or s == "N/A":
        return 0.0
    try:
        return float(s)
    except Exception:
        return 0.0


def popularity_score(item: dict[str, Any]) -> float:
    rating = float(item.get("vote_average") or 0.0)
    votes = parse_int(item.get("imdb_votes_raw"))
    metascore = parse_int(item.get("metascore_raw"))
    # Weighted score combining IMDb rating, votes and Metascore
    return rating * (1 + log(1 + votes)) + 0.02 * metascore


def filter_item(item: dict[str, Any], *, min_year: int | None = None, max_year: int | None = None, languages: set[str] | None = None, types: set[str] | None = None) -> bool:
    year_str = item.get("year") or ""
    try:
        year_int = int(str(year_str)[:4]) if year_str else None
    except Exception:
        year_int = None
    if min_year is not None and (year_int is None or year_int < min_year):
        return False
    if max_year is not None and (year_int is None or year_int > max_year):
        return False
    if languages:
        lang = (item.get("language") or "").lower()
        if not any(l.lower() in lang for l in languages):
            return False
    if types:
        t = (item.get("type") or "movie").lower()
        if t not in types:
            return False
    return True


In [8]:
# Enhanced recommend using improved helpers, filters, and scoring
from random import sample

async def recommend_enhanced(
    mood: str,
    *,
    limit: int = 6,
    types: tuple[str, ...] = ("movie",),
    min_year: int | None = None,
    max_year: int | None = None,
    pages: int = 3,
    languages: tuple[str, ...] = tuple(),
) -> list[dict[str, Any]]:
    # Expand queries similar to app's heuristic plus generic pool
    m = (mood or "").strip().lower()
    queries: list[str] = [f"{m} horror", "horror", "scary horror", "supernatural horror", "slasher horror", "zombie horror"]

    # Collect IDs from multiple pages and types
    ids: list[str] = []
    for q in queries:
        ids += await search_ids(q, types=types, pages=pages)
    ids = list(dict.fromkeys(ids))

    # Fetch full details (with plot=full) and filter
    items = await fetch_details(ids, horror_only=True)

    chosen: list[dict[str, Any]] = []
    lang_set = set(languages)
    type_set = set(t.lower() for t in types)
    for it in items:
        if not filter_item(it, min_year=min_year, max_year=max_year, languages=lang_set or None, types=type_set or None):
            continue
        it["_score"] = popularity_score(it)
        chosen.append(it)

    if not chosen:
        return []

    chosen_sorted = sorted(chosen, key=lambda x: x.get("_score", 0.0), reverse=True)
    pool = chosen_sorted[: max(10, limit * 3)]
    if len(pool) <= limit:
        return [{k: v for k, v in m.items() if k != "_score"} for m in pool][:limit]
    sampled = sample(pool, k=limit)
    return [{k: v for k, v in m.items() if k != "_score"} for m in sampled]


In [9]:
# Example A: Year range + multi-page search (movies only)
from asyncio import Future

MOOD = "atmospheric"
LIMIT = 6

# Favor modern horror: 2005-2024, search more pages for breadth
movies = run(recommend_enhanced(
    mood=MOOD,
    limit=LIMIT,
    types=("movie",),
    min_year=2005,
    max_year=2024,
    pages=4,
    languages=("english",),
))

if hasattr(movies, "__await__") or isinstance(movies, Future):
    movies = await movies

show_results(movies)


1. Untitled Horror Movie  |  Rating: 4.8  |  Released: 15 Jun 2021
   A comedy about making a horror movie. When six co-stars learn their hit TV show is about to be canceled, they decide to shoot their own film, unintentionally summoning a spirit with an affinity for vi...
   Poster: https://m.media-amazon.com/images/M/MV5BYzgyNmFmMDUtMjc5OC00OTVjLWI2NzUtMzRhNWI4MmM2MTk1XkEyXkFqcGc@._V1_SX300.jpg

2. The Amityville Horror  |  Rating: 5.9  |  Released: 15 Apr 2005
   In December 1975, George and Kathy Lutz along with their three children move into an elegant Long Island house. What they don't know is that the house was the site of a horrific mass murder a year bef...
   Poster: https://m.media-amazon.com/images/M/MV5BMzc1Njc2NDc3NV5BMl5BanBnXkFtZTYwODYyNzI3._V1_SX300.jpg

3. My Amityville Horror  |  Rating: 5.4  |  Released: 22 Sep 2012
   For the first time in 35 years, Daniel Lutz recounts his version of the infamous Amityville haunting that terrified his family in 1975. George and Ka

In [12]:
# Example B: Include series in candidates
from asyncio import Future

MOOD = "paranormal haunting"
LIMIT = 6

movies = run(recommend_enhanced(
    mood=MOOD,
    limit=LIMIT,
    types=("movie", "series"),
    min_year=1990,
    pages=3,
    languages=("english",),
))

if hasattr(movies, "__await__") or isinstance(movies, Future):
    movies = await movies

show_results(movies)


1. Eli Roth's History of Horror  |  Rating: 8.0  |  Released: 14 Oct 2018
   An in-depth look at the history and pop cultural significance of horror films.
   Poster: https://m.media-amazon.com/images/M/MV5BMWJkZjIwYzUtODc1My00MmM1LWEyZGItMDU0MzA5NzRiZWRmXkEyXkFqcGc@._V1_SX300.jpg

2. Nightmares in Red, White and Blue: The Evolution of the American Horror Film  |  Rating: 7.1  |  Released: 06 Aug 2009
   An exploration of the appeal of horror films, with interviews of many legendary directors in the genre.
   Poster: https://m.media-amazon.com/images/M/MV5BNjBhZTY1MjEtOTVlYS00N2RmLTk5ZjMtMGZjMTYyMjBlMDAyXkEyXkFqcGdeQXVyMTU0NTE4MTkz._V1_SX300.jpg

3. A Classic Horror Story  |  Rating: 5.7  |  Released: 14 Jul 2021
   Five carpoolers travel in a motorhome to reach a common destination. Night falls, and to avoid a dead animal carcass, they crash into a tree. When they come to their senses, they find themselves in th...
   Poster: https://m.media-amazon.com/images/M/MV5BYTg5ZDgxNDAtMzFkNS0

In [11]:
'finish'

'finish'

In [ ]:
import gc
del OMDB_API_KEY
gc.collect()


25